In [ ]:
# =============================================================================
# 1. IMPORT LIBRARIES
# =============================================================================
import os
import logging
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from transformers import RobertaModel, RobertaTokenizer
import sys

current_dir = os.getcwd()
two_steps_back = os.path.dirname(os.path.dirname(current_dir))
sys.path.append(two_steps_back)
from RoBERTa_classifier import RobertaTrainer, RobertaTokenizer

logging.basicConfig(level=logging.ERROR)

# =============================================================================
# 2. CONSTANTS & VARIABLES
# =============================================================================
# Device configuration
DEVICE = 'cuda' if cuda.is_available() else 'cpu'

# Data paths
BASE_DIR = os.path.dirname(os.getcwd())
TWO_FOLDERS_BACK = os.path.dirname(os.path.dirname(os.getcwd()))
DATA_FOLDER = os.path.join(TWO_FOLDERS_BACK, 'data')
DATA_PATH = os.path.join(DATA_FOLDER, 'MMCoVaR_News_Dataset.csv')

# Hyperparameters and settings
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
TEST_SIZE = 0.2
EPOCHS = 4
LEARNING_RATE = 1e-05
PADDING = "max_length"
RANDOM_STATE = 200

# Columns for text and target
TEXT_COLUMN = "clean_text"  # Make sure your DataFrame has this column; otherwise adjust accordingly.
TARGET_COLUMN = "target"

# =============================================================================
# 3. DATA UPLOAD & PREPARATION
# =============================================================================
# Load dataset
df_MMCoVaR = pd.read_csv(DATA_PATH)
# Mark all data as nonsynthetic (or update accordingly)
df_MMCoVaR['synthetic'] = False

# Prepare the DataFrame with required columns.
df_MMCoVaR_train = df_MMCoVaR[['body_text', 'reliability']].copy()
df_MMCoVaR_train.columns = [TEXT_COLUMN, TARGET_COLUMN]
df_MMCoVaR_train['synthetic'] = False

df = df_MMCoVaR_train

print("Ratio of target value:\n", df[TARGET_COLUMN].value_counts(normalize=True))


# =============================================================================
# 3. RUN ROBERTA
# =============================================================================



# Initialize the Roberta tokenizer.
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base', truncation=True, do_lower_case=True)


# Create an instance of our trainer.
trainer = RobertaTrainer(
    df=df,
    tokenizer=tokenizer,
    max_len=MAX_LEN,
    train_batch_size=TRAIN_BATCH_SIZE,
    valid_batch_size=VALID_BATCH_SIZE,
    test_size=TEST_SIZE,
    device=DEVICE,
    padding=PADDING,
    text_column=TEXT_COLUMN,
    target_column=TARGET_COLUMN,
    random_state=RANDOM_STATE,


)

# Step 2: Tokenize the data and create DataLoaders.
trainer.tokenize_data()

# Step 3: Build the model.
trainer.build_model()

# Step 4: Train the model.
trainer.train_model(epochs=EPOCHS, learning_rate=LEARNING_RATE)

# Step 5: Evaluate the model.
acc, prec, rec = trainer.evaluate_model()
print(f"Test Accuracy: {acc:.2f}%")
print(f"Test Precision: {prec * 100:.2f}%")
print(f"Test Recall: {rec * 100:.2f}%")

# Save the model and tokenizer.
output_model_file = 'pytorch_roberta_MMCoVaR.bin'
output_tokenizer_file = 'pytorch_roberta_MMCoVaR_tokenizer.bin'
output_vocab_file = 'results_ROBERTA'
directory_to_save = os.path.join(os.getcwd(), "saved_model", output_model_file)
directory_to_save_tokenizer = os.path.join(os.getcwd(), "saved_model", output_tokenizer_file)

trainer.save_model(os.path.join(os.getcwd(), "saved_model", output_model_file), os.getcwd())
tokenizer.save_vocabulary(directory_to_save, directory_to_save_tokenizer)

c:\Users\Lukag\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ratio of target value:
 target
1    0.630544
0    0.369456
Name: proportion, dtype: float64
TRAIN Dataset shape: (2074,)
TEST Dataset shape: (519,)


Training:   0%|          | 0/260 [00:00<?, ?it/s]

Step 0: Training Loss: 0.7439, Training Accuracy: 25.00%


Training:   0%|          | 1/260 [00:13<57:56, 13.42s/it]


KeyboardInterrupt: 